<a href="https://colab.research.google.com/github/soumabkargit/intro-to-machine-learning/blob/master/n_vibe_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.metrics import mean_absolute_error,mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from math import sqrt

In [35]:
#Load the Data
df = pd.read_csv('data.csv')
df.head()

,Unnamed: 0,device,rssi,Personne,Iteration,Label_pm,lat,lon
0,0,C8:DF:84:32:FA:C6,"[-74, -74, -74, -74, -74, -74, -74, -74, -74, ...",personne1,iteration1,I5,50.844861,4.373333
1,1,C8:DF:84:34:5E:68,"[-86, -86, -86, -86, -86, -86, -86, -87, -87, ...",personne1,iteration1,I5,50.844861,4.373333
2,2,C8:DF:84:28:57:C0,"[-92, -92, -92, -92, -92, -92, -92, -92, -92, ...",personne1,iteration1,I5,50.844861,4.373333
3,3,C8:DF:84:34:1B:03,"[-79, -79, -79, -79, -79, -79, -79, -79, -79, ...",personne1,iteration1,I5,50.844861,4.373333
4,4,C8:DF:84:34:1D:2A,"[-81, -81, -81, -81, -81, -81, -81, -81, -81, ...",personne1,iteration1,I5,50.844861,4.373333


In [36]:
#Data PreProcessing
len(df)
df.isnull().sum()
df = df.dropna()
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float) #.to_list()

In [37]:
temp_all_rssi_row = ""
array_rssi_row = []
temp_device = []
temp_personne = []
temp_iteration = []
temp_label_pm = []
temp_lat = []
temp_lon = []

temp_rssi_1,temp_rssi_2,temp_rssi_3,temp_rssi_4,temp_rssi_5 = [],[],[],[],[]
temp_rssi_6,temp_rssi_7,temp_rssi_8,temp_rssi_9,temp_rssi_10 = [],[],[],[],[]
temp_rssi_11,temp_rssi_12,temp_rssi_13,temp_rssi_14,temp_rssi_15 = [],[],[],[],[]
temp_rssi_16,temp_rssi_17,temp_rssi_18,temp_rssi_19,temp_rssi_20 = [],[],[],[],[]
temp_rssi_21,temp_rssi_22,temp_rssi_23,temp_rssi_24,temp_rssi_25 = [],[],[],[],[]
temp_rssi_26,temp_rssi_27,temp_rssi_28,temp_rssi_29,temp_rssi_30 = [],[],[],[],[]
temp_rssi_31,temp_rssi_32,temp_rssi_33,temp_rssi_34,temp_rssi_35 = [],[],[],[],[]
temp_rssi_36,temp_rssi_37,temp_rssi_38,temp_rssi_39,temp_rssi_40 = [],[],[],[],[]
temp_rssi_41,temp_rssi_42,temp_rssi_43,temp_rssi_44,temp_rssi_45 = [],[],[],[],[]
temp_rssi_46,temp_rssi_47,temp_rssi_48,temp_rssi_49 = [],[],[],[]

for index_df, row_lppi in df.iterrows():
  temp_array_rssi = []
  temp_all_rssi_row = row_lppi['rssi'].lstrip('[')
  temp_all_rssi_row = temp_all_rssi_row.rstrip(']')
  array_rssi_row = temp_all_rssi_row.split(',')

  if len(array_rssi_row) == 49 :
  
    row_lppi['rssi_1'],row_lppi['rssi_2'],row_lppi['rssi_3'],row_lppi['rssi_4'],row_lppi['rssi_5'] = float(array_rssi_row[0]),float(array_rssi_row[1]),float(array_rssi_row[2]),float(array_rssi_row[3]),float(array_rssi_row[4])
    row_lppi['rssi_6'],row_lppi['rssi_7'],row_lppi['rssi_8'],row_lppi['rssi_9'],row_lppi['rssi_10'] = float(array_rssi_row[5]),float(array_rssi_row[6]),float(array_rssi_row[7]),float(array_rssi_row[8]),float(array_rssi_row[9])
    row_lppi['rssi_11'],row_lppi['rssi_12'],row_lppi['rssi_13'],row_lppi['rssi_14'],row_lppi['rssi_15'] = float(array_rssi_row[10]),float(array_rssi_row[11]),float(array_rssi_row[12]),float(array_rssi_row[13]),float(array_rssi_row[14])
    row_lppi['rssi_16'],row_lppi['rssi_17'],row_lppi['rssi_18'],row_lppi['rssi_19'],row_lppi['rssi_20'] = float(array_rssi_row[15]),float(array_rssi_row[16]),float(array_rssi_row[17]),float(array_rssi_row[18]),float(array_rssi_row[19])
    row_lppi['rssi_21'],row_lppi['rssi_22'],row_lppi['rssi_23'],row_lppi['rssi_24'],row_lppi['rssi_25'] = float(array_rssi_row[20]),float(array_rssi_row[21]),float(array_rssi_row[22]),float(array_rssi_row[23]),float(array_rssi_row[24])
    row_lppi['rssi_26'],row_lppi['rssi_27'],row_lppi['rssi_28'],row_lppi['rssi_29'],row_lppi['rssi_30'] = float(array_rssi_row[25]),float(array_rssi_row[26]),float(array_rssi_row[27]),float(array_rssi_row[28]),float(array_rssi_row[29])
    row_lppi['rssi_31'],row_lppi['rssi_32'],row_lppi['rssi_33'],row_lppi['rssi_34'],row_lppi['rssi_35'] = float(array_rssi_row[30]),float(array_rssi_row[31]),float(array_rssi_row[32]),float(array_rssi_row[33]),float(array_rssi_row[34])
    row_lppi['rssi_36'],row_lppi['rssi_37'],row_lppi['rssi_38'],row_lppi['rssi_39'],row_lppi['rssi_40'] = float(array_rssi_row[35]),float(array_rssi_row[36]),float(array_rssi_row[37]),float(array_rssi_row[38]),float(array_rssi_row[39])
    row_lppi['rssi_41'],row_lppi['rssi_42'],row_lppi['rssi_43'],row_lppi['rssi_44'],row_lppi['rssi_45'] = float(array_rssi_row[40]),float(array_rssi_row[41]),float(array_rssi_row[42]),float(array_rssi_row[43]),float(array_rssi_row[44])
    row_lppi['rssi_46'],row_lppi['rssi_47'],row_lppi['rssi_48'],row_lppi['rssi_49'] = float(array_rssi_row[45]),float(array_rssi_row[46]),float(array_rssi_row[47]),float(array_rssi_row[48])

    temp_device.append(row_lppi['device'])
    temp_personne.append(row_lppi['Personne'])
    temp_iteration.append(row_lppi['Iteration'])
    temp_label_pm.append(row_lppi['Label_pm'])
    temp_lat.append(row_lppi['lat'])
    temp_lon.append(row_lppi['lon'])

    temp_rssi_1.append(row_lppi['rssi_1'])
    temp_rssi_2.append(row_lppi['rssi_2'])
    temp_rssi_3.append(row_lppi['rssi_3'])
    temp_rssi_4.append(row_lppi['rssi_4'])
    temp_rssi_5.append(row_lppi['rssi_5'])
    temp_rssi_6.append(row_lppi['rssi_6'])
    temp_rssi_7.append(row_lppi['rssi_7'])
    temp_rssi_8.append(row_lppi['rssi_8'])
    temp_rssi_9.append(row_lppi['rssi_9'])
    temp_rssi_10.append(row_lppi['rssi_10'])
    temp_rssi_11.append(row_lppi['rssi_11'])
    temp_rssi_12.append(row_lppi['rssi_12'])
    temp_rssi_13.append(row_lppi['rssi_13'])
    temp_rssi_14.append(row_lppi['rssi_14'])
    temp_rssi_15.append(row_lppi['rssi_15'])
    temp_rssi_16.append(row_lppi['rssi_16'])
    temp_rssi_17.append(row_lppi['rssi_17'])
    temp_rssi_18.append(row_lppi['rssi_18'])
    temp_rssi_19.append(row_lppi['rssi_19'])
    temp_rssi_20.append(row_lppi['rssi_20'])
    temp_rssi_21.append(row_lppi['rssi_21'])
    temp_rssi_22.append(row_lppi['rssi_22'])
    temp_rssi_23.append(row_lppi['rssi_23'])
    temp_rssi_24.append(row_lppi['rssi_24'])
    temp_rssi_25.append(row_lppi['rssi_25'])
    temp_rssi_26.append(row_lppi['rssi_26'])
    temp_rssi_27.append(row_lppi['rssi_27'])
    temp_rssi_28.append(row_lppi['rssi_28'])
    temp_rssi_29.append(row_lppi['rssi_29'])
    temp_rssi_30.append(row_lppi['rssi_30'])
    temp_rssi_31.append(row_lppi['rssi_31'])
    temp_rssi_32.append(row_lppi['rssi_32'])
    temp_rssi_33.append(row_lppi['rssi_33'])
    temp_rssi_34.append(row_lppi['rssi_34'])
    temp_rssi_35.append(row_lppi['rssi_35'])
    temp_rssi_36.append(row_lppi['rssi_36'])
    temp_rssi_37.append(row_lppi['rssi_37'])
    temp_rssi_38.append(row_lppi['rssi_38'])
    temp_rssi_39.append(row_lppi['rssi_39'])
    temp_rssi_40.append(row_lppi['rssi_40'])
    temp_rssi_41.append(row_lppi['rssi_41'])
    temp_rssi_42.append(row_lppi['rssi_42'])
    temp_rssi_43.append(row_lppi['rssi_43'])
    temp_rssi_44.append(row_lppi['rssi_44'])
    temp_rssi_45.append(row_lppi['rssi_45'])
    temp_rssi_46.append(row_lppi['rssi_46'])
    temp_rssi_47.append(row_lppi['rssi_47'])
    temp_rssi_48.append(row_lppi['rssi_48'])
    temp_rssi_49.append(row_lppi['rssi_49'])

In [38]:
all_data = {'device' : temp_device,'personne' : temp_personne, 'iteration' : temp_iteration, 'label' : temp_label_pm, 'lat' : temp_lat, 'lon' : temp_lon,
            'rssi1' : temp_rssi_1, 'rssi2' : temp_rssi_2, 'rssi3' : temp_rssi_3, 'rssi4' : temp_rssi_4,
            'rssi5' : temp_rssi_5, 'rssi6' : temp_rssi_6, 'rssi7' : temp_rssi_7, 'rssi8' : temp_rssi_8,
            'rssi9' : temp_rssi_9, 'rssi10' : temp_rssi_10, 'rssi11' : temp_rssi_11,'rssi12' : temp_rssi_12,
            'rssi13' : temp_rssi_13, 'rssi14' : temp_rssi_14, 'rssi15' : temp_rssi_15, 'rssi16' : temp_rssi_16,
            'rssi17' : temp_rssi_17,'rssi18' : temp_rssi_18,'rssi19' : temp_rssi_19,'rssi20' : temp_rssi_20,
            'rssi21' : temp_rssi_21, 'rssi22' : temp_rssi_22,'rssi23' : temp_rssi_23,'rssi24' : temp_rssi_24,
            'rssi25' : temp_rssi_25,'rssi26' : temp_rssi_26,'rssi27' : temp_rssi_27,'rssi28' : temp_rssi_28,
            'rssi29' : temp_rssi_29,'rssi30' : temp_rssi_30,'rssi31' : temp_rssi_31,'rssi32' : temp_rssi_32,
            'rssi33' : temp_rssi_33,'rssi34' : temp_rssi_34,'rssi35' : temp_rssi_35,'rssi36' : temp_rssi_36,
            'rssi37' : temp_rssi_37,'rssi38' : temp_rssi_38,'rssi39' : temp_rssi_39,'rssi40' : temp_rssi_40,
            'rssi41' : temp_rssi_41,'rssi42' : temp_rssi_42,'rssi43' : temp_rssi_43,'rssi44' : temp_rssi_44,
            'rssi45' : temp_rssi_45, 'rssi46' : temp_rssi_46, 'rssi47' : temp_rssi_47,'rssi48' : temp_rssi_48,
            'rssi49' : temp_rssi_49}
df  = pd.DataFrame(data= all_data)

In [39]:
df

,device,personne,iteration,label,lat,lon,rssi1,rssi2,rssi3,rssi4,rssi5,rssi6,rssi7,rssi8,rssi9,rssi10,rssi11,rssi12,rssi13,rssi14,rssi15,rssi16,rssi17,rssi18,rssi19,rssi20,rssi21,rssi22,rssi23,rssi24,rssi25,rssi26,rssi27,rssi28,rssi29,rssi30,rssi31,rssi32,rssi33,rssi34,rssi35,rssi36,rssi37,rssi38,rssi39,rssi40,rssi41,rssi42,rssi43,rssi44,rssi45,rssi46,rssi47,rssi48,rssi49
0,C8:DF:84:32:FA:C6,personne1,iteration1,I5,50.844861,4.373333,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-75.0,-75.0,-75.0,-75.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-74.0,-79.0,-79.0,-79.0,-79.0,-79.0,-80.0,-80.0,-80.0,-80.0,-80.0,-78.0,-78.0,-78.0,-78.0,-78.0,-78.0,-78.0,-78.0,-78.0,-78.0,-77.0
1,C8:DF:84:34:5E:68,personne1,iteration1,I5,50.844861,4.373333,-86.0,-86.0,-86.0,-86.0,-86.0,-86.0,-86.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-89.0,-89.0,-89.0,-89.0,-89.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-89.0,-89.0,-89.0
2,C8:DF:84:28:57:C0,personne1,iteration1,I5,50.844861,4.373333,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0,-92.0
3,C8:DF:84:34:1B:03,personne1,iteration1,I5,50.844861,4.373333,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-79.0,-80.0,-80.0,-80.0,-80.0,-84.0,-84.0,-84.0,-84.0,-84.0,-84.0,-84.0,-84.0,-84.0,-84.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0,-91.0
4,C8:DF:84:34:1D:2A,personne1,iteration1,I5,50.844861,4.373333,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-81.0,-89.0,-89.0,-89.0,-89.0,-89.0,-89.0,-89.0,-89.0,-83.0,-83.0,-83.0,-83.0,-83.0,-87.0,-82.0,-82.0,-83.0,-80.0,-80.0,-85.0,-85.0,-85.0,-85.0,-85.0,-85.0,-88.0,-88.0,-88.0,-88.0,-88.0,-88.0,-88.0,-88.0,-88.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15919,C8:DF:84:28:F7:1F,personne1,iteration1,C5,50.845083,4.373667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0,-97.0
15920,C8:DF:84:34:1E:E0,personne1,iteration1,C5,50.845083,4.373667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0,-87.0
15921,C8:DF:84:28:55:91,personne1,iteration1,C5,50.845083,4.373667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-94.0,-94.0,-94.0,-94.0,-94.0
15922,C8:DF:84:28:F5:86,personne1,iteration1,C5,50.845083,4.373667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-98.0,-98.0


In [40]:
df.to_csv('data_general.csv')
df.to_excel("data_general.xlsx") 

In [41]:
X = df[['lat','lon','rssi1','rssi2','rssi3','rssi4','rssi5','rssi6','rssi7','rssi8','rssi9','rssi10',
       'rssi11','rssi12','rssi13','rssi14','rssi15','rssi16','rssi17','rssi18','rssi19','rssi20','rssi21',
       'rssi22','rssi23','rssi24','rssi25','rssi26','rssi27','rssi28','rssi29','rssi30','rssi31','rssi32',
       'rssi33','rssi34','rssi35','rssi36','rssi37','rssi38','rssi39','rssi40','rssi41','rssi42','rssi43',
       'rssi44','rssi45','rssi46','rssi47','rssi48','rssi49']].values

Y = df['label'].values

In [42]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

print('X_train :',X_train.shape)
print('X_test :',X_test.shape)
print('y_train :',y_train.shape)
print('y_test :',y_test.shape)

X_train : (11146, 51)
X_test : (4778, 51)
y_train : (11146,)
y_test : (4778,)


In [43]:
# create the model
sgb_classifier = GradientBoostingClassifier()

In [44]:
# fit the model on the training set
sgb_classifier.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [46]:
# make predictions on the test set
predictions = sgb_classifier.predict(X_test)

In [47]:
predictions

array(['A3', 'K1', 'G2', ..., 'D2', 'M6', 'E3'], dtype=object)

In [48]:
# evaluate the model performance using accuracy metric
print("Accuracy: %.2f" % accuracy_score(y_test, predictions))

Accuracy: 1.00


In [49]:
pred_df = pd.DataFrame(y_test,columns=['Test Y'])
pred_df
predictions = pd.Series(predictions.reshape(4778,))
predictions
pred_df = pd.concat([pred_df,predictions],axis=1)
pred_df.columns = ['Test Y','Model Predictions']
pred_df

,Test Y,Model Predictions
0,A3,A3
1,K1,K1
2,G2,G2
3,N3,N3
4,N2,N2
...,...,...
4773,D2,D2
4774,A4,A4
4775,D2,D2
4776,M6,M6
